In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from util import *
from h5py import File
import glob
from tqdm.notebook import tqdm

In [8]:
def normalize(image):
    image = (image - image.min()) / (image.max() - image.min() + 1e-8)
    return image

In [3]:
with open('data/BraTS2019/val.list', 'r') as f:
    tmp_list = f.readlines() 
val_cases = [item.replace("\n", "") for item in tmp_list]
val_cases

['BraTS2019_00307.h5',
 'BraTS2019_00023.h5',
 'BraTS2019_00158.h5',
 'BraTS2019_00295.h5',
 'BraTS2019_00240.h5',
 'BraTS2019_00219.h5',
 'BraTS2019_00305.h5',
 'BraTS2019_00108.h5',
 'BraTS2019_00130.h5',
 'BraTS2019_00267.h5',
 'BraTS2019_00311.h5',
 'BraTS2019_00007.h5',
 'BraTS2019_00133.h5',
 'BraTS2019_00167.h5',
 'BraTS2019_00159.h5',
 'BraTS2019_00301.h5',
 'BraTS2019_00286.h5',
 'BraTS2019_00145.h5',
 'BraTS2019_00104.h5',
 'BraTS2019_00057.h5']

In [12]:
os.makedirs('data/BraTS2019/data/val_slices', exist_ok=True)

In [13]:
root = 'data/BraTS2019/data/'
for case in val_cases:
    path = os.path.join(root, case)
    assert os.path.exists(path), f'{path} not exists!'
    f = File(path, 'r')
    im3d = f["image"][:]
    lb3d = f["label"][:]
    for i in range(im3d.shape[0]):
        slice_name = root + 'val_slices/' + case.split('.')[0] + '_' + str(i).zfill(3) + '.h5'
        sf = File(slice_name, 'w')
        sf.create_dataset("image", data=normalize(im3d[i]), compression="gzip")
        sf.create_dataset("label", data=normalize(lb3d[i]), compression="gzip")
        sf.close()

In [21]:
train_slices = sorted(glob.glob(root + 'slices/*'))
for s in tqdm(train_slices):
    tf = File(s, 'r')
    nf = File(root + 'train_slices/' + os.path.basename(s), 'w')
    im = normalize(tf['image'][:])
    nf.create_dataset('image', data=im, compression='gzip')
    nf.create_dataset('label', data=tf['label'][:], compression='gzip')
    tf.close()
    nf.close()

  0%|          | 0/45967 [00:00<?, ?it/s]

In [26]:
new_train_slices = sorted(glob.glob(root+'val_slices/*'))
with open('data/BraTS2019/val_slices.list', 'w') as f:
    for l in new_train_slices:
        f.write(os.path.basename(l))
        f.write('\n')

In [7]:
root = 'data/BraTS2019/data/'
train_slices = sorted(glob.glob(root + 'val_slices/*'))
cnt = 0
for s in tqdm(train_slices):
    tf = File(s, 'r')
    img = tf["image"][:]
    print(img.max())
    print(img.min())
    print(img.mean())
    cnt += 1
    if cnt > 20:
        break
    tf.close()
cnt

  0%|          | 0/2762 [00:00<?, ?it/s]

0.99999964
0.0
8.1093574e-05
1.0
0.0
0.010949727
1.0
0.0
0.03808865
1.0
0.0
0.060700677
1.0
0.0
0.0665988
1.0
0.0
0.07020764
1.0
0.0
0.07451042
1.0
0.0
0.07847209
1.0
0.0
0.084911786
1.0
0.0
0.09038725
1.0
0.0
0.10566078
1.0
0.0
0.11280403
1.0
0.0
0.12240403
1.0
0.0
0.11154427
1.0
0.0
0.11660387
1.0
0.0
0.13740632
1.0
0.0
0.14187573
1.0
0.0
0.14919287
1.0
0.0
0.1717606
1.0
0.0
0.1717318
1.0
0.0
0.15659624


21